In [1]:
import pandas as pd
import numpy as np
import os
import openpyxl

In [2]:
#Base de otimização de rotas

base = pd.read_excel("Otimização de rotas.xlsx",sheet_name='Trajeto Geral (Equ x Base Faz)', skiprows=6).drop(['Unnamed: 0','Unnamed: 1'],axis=1)

In [3]:
#Capacidade por equipe
capacidade_equipe = pd.read_excel('Cadastro equipes e demandas.xlsx', sheet_name='Capacidade por equipe')

In [4]:
#Base de demandas

demanda = pd.read_excel('Cadastro equipes e demandas.xlsx', sheet_name='Demanda por fazenda', dtype={'ID Fazenda': 'object', 'Mês de Demanda': 'datetime64[ns]'})

In [5]:
#demanda 
demanda.head(5)

,ID Fazenda,Fazenda,Área Avaliada,Mês avaliação,Data de Avaliação
0,0001,MAMEDINA,1395.99,2024-04-01,2024-04-08 00:00:00
1,0003,UNIÃO,748.94,2024-03-01,2024-03-24 00:00:00
2,0004,SÃO LUIZ,196.08,2024-04-01,2024-04-03 00:00:00
3,0005,SANTA RITA,164.06,2024-04-01,2024-04-09 00:00:00
4,0006,LIMEIRA,820.61,2024-03-01,2024-03-21 04:06:51


In [6]:
#Função de máscara

def mascara(df, coluna,n_digitos):
    df[coluna] = df[coluna].astype(str)
    df[coluna] = str("000") + df[coluna]
    df[coluna] = df[coluna].astype(str).str[-n_digitos:] 
    return df

In [7]:
#Data frame de equipes e siglas
equipes = [ i for  i in base['COD'].str[7:].unique()]
equipes_s = [ i for  i in base['Equipe'].unique()]


equipes_base = pd.DataFrame(
    { 'Equipe': equipes,
     'Equipe sigla' : equipes_s
    }
)

equipes_base.head(len(equipes))

,Equipe,Equipe sigla
0,Lençóis Paulista,A
1,Duartina,B
2,Garça,C
3,Pirajuí,D
4,Bauru,E
5,Getulina,G
6,Cabrália Paulista,H


In [8]:
#Ajustando capacidade da equipe 

capacidade_equipe['Gordura'] = 0.05

capacidade_equipe['Capacidade ajustada'] = (capacidade_equipe['Gordura'] * capacidade_equipe['Capacidade']) + capacidade_equipe['Capacidade']

capacidade_equipe = capacidade_equipe.merge(equipes_base, on='Equipe', how='left')

capacidade_equipe.head(6)

,Equipe,Capacidade,Distancia,Mês Operacional,Gordura,Capacidade ajustada,Equipe sigla
0,Lençóis Paulista,7400,5000,2024-04-01,0.05,7770.0,A
1,Duartina,7400,5000,2024-04-01,0.05,7770.0,B
2,Garça,7400,5000,2024-04-01,0.05,7770.0,C
3,Pirajuí,7400,5000,2024-04-01,0.05,7770.0,D
4,Bauru,7400,5000,2024-04-01,0.05,7770.0,E
5,Getulina,7400,5000,2024-04-01,0.05,7770.0,G


In [9]:
#Demanda por fazenda mês

demanda = mascara(demanda,'ID Fazenda', 4)

demanda.head(5)

,ID Fazenda,Fazenda,Área Avaliada,Mês avaliação,Data de Avaliação
0,0001,MAMEDINA,1395.99,2024-04-01,2024-04-08 00:00:00
1,0003,UNIÃO,748.94,2024-03-01,2024-03-24 00:00:00
2,0004,SÃO LUIZ,196.08,2024-04-01,2024-04-03 00:00:00
3,0005,SANTA RITA,164.06,2024-04-01,2024-04-09 00:00:00
4,0006,LIMEIRA,820.61,2024-03-01,2024-03-21 04:06:51


In [10]:
base = base.sort_values(['Área','Trajeto (Km)'] , ascending=[False,False])
base = mascara(base,'ID Fazenda',4)
base.head(5)

,Equipe,Cidade origem,ID Fazenda,Projeto,Área,COD,Trajeto (Km)
5612,G,Getulina,2081,FLECHA AZUL,7116.95,2081 - Getulina,233.653679
5609,C,Garça,2081,FLECHA AZUL,7116.95,2081 - Garça,184.097420
5610,D,Pirajuí,2081,FLECHA AZUL,7116.95,2081 - Pirajuí,165.009948
5608,B,Duartina,2081,FLECHA AZUL,7116.95,2081 - Duartina,152.596125
5613,H,Cabrália Paulista,2081,FLECHA AZUL,7116.95,2081 - Cabrália Paulista,142.390128


In [11]:
#Encontrando a área das fazendas
area_fazendas = base[['ID Fazenda','Área', 'Projeto']].drop_duplicates()

area_fazendas.head(5)


,ID Fazenda,Área,Projeto
5612,2081,7116.95,FLECHA AZUL
4821,0967,6844.42,MARUQUE
2429,0378,4864.71,USINA PAREDÃO
5066,1110,4498.55,PIRACEMA
4919,1050,3705.50,RIO CLARO


In [12]:
#Pivotar a tabela para encontrar a distancia entre as fazendas e ter os trajetos em Km 

distancias_fazendas = base.pivot_table(
    index='ID Fazenda',
    columns='Equipe',
    values='Trajeto (Km)',
    aggfunc='sum',
    )

distancias_fazendas.head(5)

Equipe,A,B,C,D,E,G,H
ID Fazenda,,,,,,,
0001,15.396001,79.279400,110.671138,91.583666,36.688475,160.227397,69.073404
0003,64.514186,29.064904,73.350765,88.983186,40.215705,143.080911,18.858908
0004,33.164475,76.816747,121.102608,120.896143,66.000951,190.832754,66.610751
0005,33.644730,71.838997,116.124858,118.185870,63.290679,185.855005,61.633001
0006,82.470092,45.422067,67.801008,28.687512,41.195255,97.331244,54.052974


In [13]:
#Organizando uma base para realizar avaliações

distancias_fazendas = distancias_fazendas.merge(area_fazendas, left_on='ID Fazenda', right_on='ID Fazenda', how='left')

distancias_fazendas = distancias_fazendas.sort_values(['Área'], ascending=[False])


In [14]:
def menores_distancias(dataframe, columns, iteracoes):
    set_equipes = set(columns)

    for i in range(1, iteracoes + 1):
        if i == 1:
            # Primeira iteração: calcular menor distância e equipe mais próxima
            dataframe[f'menor distancia {i}'] = dataframe[columns].min(axis=1)
            dataframe[f'equipe mais proxima {i}'] = dataframe[columns].idxmin(axis=1)
        else:
            # Iterações subsequentes: calcular menor distância e equipe mais próxima excluindo as equipes anteriores
            dataframe[f'set_atualizado_{i}'] = dataframe.apply(
                lambda row: set_equipes - set([row[f'equipe mais proxima {j}'] for j in range(1, i)]), 
                axis=1
            )

            def calcular_menor_distancia(row):
                equipes_remanescentes = list(row[f'set_atualizado_{i}'])
                return row[equipes_remanescentes].min()

            def equipe_menor_distancia(row):
                equipes_remanescentes = list(row[f'set_atualizado_{i}'])
                return row[equipes_remanescentes].idxmin()

            dataframe[f'menor distancia {i}'] = dataframe.apply(calcular_menor_distancia, axis=1)
            dataframe[f'equipe mais proxima {i}'] = dataframe.apply(equipe_menor_distancia, axis=1)

    return dataframe

In [15]:
# def menores_distancias(dataframe, columns):
#     dataframe['menor distancia'] = dataframe[columns].min(axis=1)
#     dataframe['equipe mais proxima'] = dataframe[columns].idxmin(axis=1)

#     set_equipes = set(columns)
      
#     dataframe['set_atualizado'] = dataframe.apply(
#         lambda row: set_equipes - {row['equipe mais proxima']}, 
#         axis=1
#     )

#     def calcular_menor_distancia_2(row):
#         equipes_remanescentes = list(row['set_atualizado'])
#         return row[equipes_remanescentes].min()


#     def equipe_menor_distancia_2(row):
#         equipes_remanescentes = list(row['set_atualizado'])
#         return row[equipes_remanescentes].idxmin()
    


#     dataframe['menor distancia 2'] = dataframe.apply(calcular_menor_distancia_2, axis=1)
#     dataframe['equipe mais proxima 2'] = dataframe.apply(equipe_menor_distancia_2,axis=1)


#     dataframe['set_atualizado_2'] = dataframe.apply(
#         lambda row: (set_equipes - {row['equipe mais proxima']}) - {row['equipe mais proxima 2']}, 
#         axis=1
#     )

#     def calcular_menor_distancia3(row):
#         equipes_remanescentes = list(row['set_atualizado_2'])
#         return row[equipes_remanescentes].min()

#     def equipe_menor_distancia_3(row):
#         equipes_remanescentes = list(row['set_atualizado_2'])
#         return row[equipes_remanescentes].idxmin()


#     dataframe['menor distancia 3'] = dataframe.apply(calcular_menor_distancia3, axis=1)
#     dataframe['equipe mais proxima 3'] = dataframe.apply(equipe_menor_distancia_3,axis=1)





#     return dataframe

In [16]:
distancias_fazendas = menores_distancias(distancias_fazendas, equipes_s,4)

In [17]:

avaliar = area_fazendas.merge(demanda[['ID Fazenda', 'Área Avaliada', 'Mês avaliação']], left_on='ID Fazenda', right_on='ID Fazenda', how='left')


avaliar = avaliar.merge(distancias_fazendas[['ID Fazenda','menor distancia 1', 'equipe mais proxima 1','menor distancia 2', 'equipe mais proxima 2','menor distancia 3', 'equipe mais proxima 3', 'menor distancia 4', 'equipe mais proxima 4']], left_on='ID Fazenda', right_on='ID Fazenda', how='left')

avaliar = avaliar[pd.notna(avaliar['Mês avaliação'])]

avaliar

,ID Fazenda,Área,Projeto,Área Avaliada,Mês avaliação,menor distancia 1,equipe mais proxima 1,menor distancia 2,equipe mais proxima 2,menor distancia 3,equipe mais proxima 3,menor distancia 4,equipe mais proxima 4
0,2081,7116.95,FLECHA AZUL,1473.70,2024-03-01,106.761611,A,110.114757,E,142.390128,H,152.596125,B
2,0378,4864.71,USINA PAREDÃO,2755.85,2024-03-01,74.365240,G,79.433593,C,99.607730,D,119.336620,B
5,2086,3234.43,TRÊS LAGOAS II - ANGATUBA,2123.01,2024-03-01,120.535757,A,161.350949,E,192.902865,H,201.472534,B
6,0526,3049.22,SANTA IZABEL II - AGUDOS,1905.91,2024-04-01,48.719439,A,51.653666,H,60.085001,E,61.859662,B
7,0487,3018.89,MORRO DE OURO,2357.75,2024-03-01,72.440649,A,113.255841,E,145.640770,H,155.846766,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,2050,14.35,ESTÂNCIA MIRANTE,3.03,2024-04-01,9.646346,G,73.314931,C,74.963681,D,117.360907,B
941,2121,11.92,SANTA IZABEL III - BARIRI,9.88,2024-03-01,81.952760,E,85.252196,A,120.880714,H,128.154883,B
942,2036,10.24,SÃO JOÃO XI - ITIRAPINA,4.31,2024-03-01,151.185876,A,161.734666,E,194.010038,H,204.216035,B
943,0386,6.01,CORVO BRANCO II,5.79,2024-04-01,4.975775,A,41.465875,E,73.052560,H,83.258557,B
